## Observations and Insights 

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np


# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)


# Combine the data into a single datase
combo_metadata_results = pd.merge(study_results, mouse_metadata, on="Mouse ID", how="outer")
combo_metadata_results.head()


,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [4]:
# Checking the number of mice in the DataFrame.
orig_number_of_mice = len(combo_metadata_results["Mouse ID"].unique())
orig_number_of_mice

249

In [5]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
#duplicate_mice_id_tpoint = combo_metadata_results.duplicated("Mouse ID")
#duplicate_mice_id_tpoint
#data["First Name"].duplicated(keep = False) 
#duplicate_mice_id_tpoint = combo_metadata_results[combo_metadata_results.duplicated(["Mouse ID"], keep = "first")]
#duplicate_mice_id_tpoint.info()
duplicate_mice_id_tpoint = combo_metadata_results.loc[combo_metadata_results.duplicated(subset=['Mouse ID', 'Timepoint',]),'Mouse ID'].unique()
duplicate_mice_id_tpoint

array(['g989'], dtype=object)

In [6]:
# Optional: Get all the data for the duplicate mouse ID. 


In [7]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
new_combo_metadata_df = combo_metadata_results[combo_metadata_results['Mouse ID'] != 'g989']
new_combo_metadata_df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1880 entries, 0 to 1892
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Mouse ID            1880 non-null   object 
 1   Timepoint           1880 non-null   int64  
 2   Tumor Volume (mm3)  1880 non-null   float64
 3   Metastatic Sites    1880 non-null   int64  
 4   Drug Regimen        1880 non-null   object 
 5   Sex                 1880 non-null   object 
 6   Age_months          1880 non-null   int64  
 7   Weight (g)          1880 non-null   int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 132.2+ KB


In [8]:
# Checking the number of mice in the clean DataFrame.
new_number_mice = len(new_combo_metadata_df["Mouse ID"].unique())
new_number_mice

248

## Summary Statistics

In [9]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method is the most straighforward, creating multiple series and putting them all together at the end.

In [21]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
group_by_regimen = new_combo_metadata_df.groupby("Drug Regimen")
group_by_regimen
#tumor_vol = group_by_regimen["Tumor Volume (mm3)"]
#tumor_vol
mean_tumor_vol = group_by_regimen["Tumor Volume (mm3)"].mean()

median_tumor_vol = group_by_regimen["Tumor Volume (mm3)"].median()


std_tumor_vol = group_by_regimen["Tumor Volume (mm3)"].std()

sem_tumor_vol = group_by_regimen["Tumor Volume (mm3)"].sem()

tumor_vol_summary_stats = pd.DataFrame({"Mean Tumor Volume":     mean_tumor_vol,
                                        "Median Tumor Volume":   median_tumor_vol,
                                        "Tumor Volume Variance": var_tumor_vol,
                                        "Tumor Volume Std Dev":  std_tumor_vol,
                                        "Tumor Volume Std Err":  sem_tumor_vol})
tumor_vol_summary_stats

,Mean Tumor Volume,Median Tumor Volume,Tumor Volume Variance,Tumor Volume Std Dev,Tumor Volume Std Err
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


## Bar Plots

In [ ]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pandas. 

In [ ]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pyplot.



## Pie Plots

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the most promising treatment regimens. Calculate the IQR and quantitatively determine if there are any potential outliers. 


In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest

## Line and Scatter Plots

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen



## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
